# File 02/2

# DESCRIPTION:
## INPUT FILE:
"./OUTPUTS/dataframe_02_1.csv"
## OUTPUT FILE:
"./'OUTPUTS/dataframe_02_2.csv'"
## OPTIONAL OUTPUT_FILE 
"./OUTPUTS/VERBS_by_REGIONxCENTURY.xlsx" -> Table; is not needed afterwards

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("OUTPUTS/dataframe_02_1.csv",  dtype={"Russian Translation": "string", "English Translation": "string"})
df.head(2)

/var/folders/ps/86qydyw53xj72p_g1g25hfx00000gn/T/ipykernel_15012/1043706187.py:1: DtypeWarning: Columns (17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("OUTPUTS/dataframe_02_1.csv",  dtype={"Russian Translation": "string", "English Translation": "string"})


,Unnamed: 0,File,Text Title,Language,Sentence ID,Token ID,Form,Lemma,POS,Morphology,...,Relation,Presentation After,Russian Translation,English Translation,Type,century,exact,lang,source,place
0,0,mst,Mstislav’s letter,orv,189407,2157773,Се,се,I-,---------n,...,voc,,"вот, это","behold, here is",OR,12,1130,OR,NaN,Novgorod
1,1,mst,Mstislav’s letter,orv,189407,2157774,азъ,азъ,Pp,1s---mn--i,...,sub,,я,I,OR,12,1130,OR,NaN,Novgorod


## Convert column "exact" to 4-digit year values (YYYY) (keep NaN if no info)

In [3]:
df["exact"].value_counts().head(10)

exact
1377         32768
1377         23957
1672-1675    22835
1417.0       16121
1417          4240
1490          2487
1663.0        1835
1229.0        1421
1266.0         423
1579.0         421
Name: count, dtype: int64

In [4]:
df["exact"] = (
    df["exact"]
    .astype("string")      # type to strings first 
    .str.slice(0, 4)       # take first 4 chars 
    .astype("Int64")       # pandas nullable integer
)

In [5]:
df["exact"].value_counts().head(10)

exact
1377    56725
1672    22835
1417    20361
1490     2487
1663     1835
1229     1421
1266      423
1579      421
1100      386
1160      357
Name: count, dtype: Int64

In [6]:
df.columns

Index(['Unnamed: 0', 'File', 'Text Title', 'Language', 'Sentence ID',
       'Token ID', 'Form', 'Lemma', 'POS', 'Morphology', 'Head ID', 'Relation',
       'Presentation After', 'Russian Translation', 'English Translation',
       'Type', 'century', 'exact', 'lang', 'source', 'place'],
      dtype='object')

In [7]:
# Check for existing places in df 
list_place = list(df.place.unique())
df.place.unique()

array(['Novgorod', 'Pskov', 'Unknown', 'Staraja_Russa', 'Moscow',
       'Byzantine_Empire', 'Nizhny_Novgorod', 'Smolensk', 'Kiev',
       'Latvia', 'NorthernRussia', 'Tver'], dtype=object)

### ERROR CHECKING: all columns in df["place"] should have valid values

In [8]:
# Filter for place == NaN 
missing_place = df.loc[df['place'].isna(), ['File', 'Text Title']]
missing_place = missing_place.drop_duplicates()

assert len(missing_place) == 0, "ERROR: df['place'] still contains NaN values"
# Execute if missing values exist 
# print(missing_place.to_string(index=False))

## Create Mapping for "place" (in DataFrame) 

In [9]:
# Map the cities with their corresponding regions 
# The regions are base on languages

# PLACE    :    REGION
region_map = {
    'Tver':           'East Slavic',
    'Moscow':         'East Slavic',
    'Novgorod':       'East Slavic',
    'Pskov':          'East Slavic',
    'Staraja_Russa':  'East Slavic',
    'NorthernRussia': 'East Slavic',
    'Kiev':           'East Slavic',
    'Nizhny_Novgorod':'East Slavic',
    'Smolensk':       'East Slavic',
    'Byzantine_Empire':'Byzantine_Empire',
    'Latvia':         'Latvia',
    None:             'Unknown', # if df["place"] is None
    'Unknown':        'Unknown' # if place is unknown, so is region 
}

In [10]:
# Create a list of unique places from df["place"]
list_place = list(df.place.unique())

# Keys in region_map, which are missing in list_place
missing_in_places = [region for region in region_map.keys() 
                     if region not in list_place]

if missing_in_places != [None]:
    print("region_map keys missing in list_place:", missing_in_places)
    raise AssertionError("ERROR: Missing places exist")

missing_in_map = [place for place in list_place 
                  if place not in region_map.keys()]

if missing_in_map: # if list is not empty 
    print("list_place entries missing in region_map:", missing_in_map)
    raise AssertionError(f"ERROR: Missing places:\n\n{missing_in_map}")

## insert column "region" to DataFrame 

In [11]:
# Map region vals via "place" col (NaN is kept)
region_series = df["place"].map(region_map)

# Set unmapped Vals (including real NaNs) to "Unknown"
region_series = region_series.fillna("Unknown")

# Get idx of col "place" and insert mapped value to col "region_map"
place_idx = df.columns.get_loc("place")
df.insert(place_idx+1, "region", region_series)


## Validation for  df["region"]:

In [12]:
# assert column exists
assert "place" in df.columns, "ERROR: column 'places' does not exist"

# assert column has no NA / None
assert df["place"].notna().all(), "ERROR: column 'places' contains NA/None"

In [13]:
# Validation for  df["region"]:
assert not df["region"].isna().any(), "ERROR: region contains NaN values"

In [14]:
df[:2]

,Unnamed: 0,File,Text Title,Language,Sentence ID,Token ID,Form,Lemma,POS,Morphology,...,Presentation After,Russian Translation,English Translation,Type,century,exact,lang,source,place,region
0,0,mst,Mstislav’s letter,orv,189407,2157773,Се,се,I-,---------n,...,,"вот, это","behold, here is",OR,12,1130,OR,NaN,Novgorod,East Slavic
1,1,mst,Mstislav’s letter,orv,189407,2157774,азъ,азъ,Pp,1s---mn--i,...,,я,I,OR,12,1130,OR,NaN,Novgorod,East Slavic


In [15]:
df.to_csv('OUTPUTS/dataframe_02_2.csv', index=False)

######################################################################
# From here on: values will not be needed for oncoming files 
######################################################################

In [16]:
df_century_value_counts_before_filter = df.century.value_counts()
print(df_century_value_counts_before_filter)

century
12    93208
15    42789
11    25932
17    25924
16    23880
13    20447
14     3095
Name: count, dtype: int64


# uniq_vals 

In [17]:
def get_unique_column_values(df, column):
    return  df[column].unique().tolist()
    
uniq_vals = get_unique_column_values(df, "File")
print(uniq_vals)

['mst', 'mstislav-col', 'birchbark', 'pskov', 'const', 'luk-koloc', 'lav', 'smol-pol-lit', 'nov-sin', 'avv', 'kiev-hyp', 'peter', 'vest-kur', 'spi', 'zadon', 'rusprav', 'pskov-ivan', 'rig-smol1281', 'drac', 'sergrad', 'nov-list', 'ostromir-col', 'varlaam', 'afnik', 'dux-grjaz', 'ust-vlad', 'riga-goth', 'domo', 'usp-sbor', 'schism', 'nov-marg', 'suz-lav', 'novgorod-jaroslav', 'pvl-hyp']


In [18]:
def count_column_value_counts_verbs(df, column, column_value, column2, column_value2):
    """
    Desc:
        - Filter df so that only columns with 
        df[column] == column_value AND  df[column2] == column_value2 remain
        - Group by "Text Title" and "File", then count these combinations

    Returns: 
        -  For each combination of ("Text Title" and "File"): sum 
    """
    # 1. Filter: beide Vergleiche in Klammern, exakt POS == "V-"
    df_subset_verbs = df.loc[
        (df[column] == column_value) &
        (df[column2] == column_value2),
        :
    ]

    # 2. Gruppieren und zählen
    combi_counts_verbs = (
        df_subset_verbs
        .groupby(["Text Title", "File"])
        .size()
        .reset_index(name="count_verbs")
    )

    # 3. Ausgabe
    print(f"\nSUMME aller Tokens in dieser Teilmenge ({column} == '{column_value}' und {column2} == '{column_value2}'):")
    print(combi_counts_verbs["count_verbs"].sum())

    return combi_counts_verbs

# Create various tables containing info on texts and word frequencies

In [19]:
df.columns

Index(['Unnamed: 0', 'File', 'Text Title', 'Language', 'Sentence ID',
       'Token ID', 'Form', 'Lemma', 'POS', 'Morphology', 'Head ID', 'Relation',
       'Presentation After', 'Russian Translation', 'English Translation',
       'Type', 'century', 'exact', 'lang', 'source', 'place', 'region'],
      dtype='object')

In [20]:
def x_col_times_y_col(df, col_file, col_cent):
    """
    Return table with combinations of:
        col "File" * col "century"
    """
    # 1. Group and count combinations
    counts_ = df.groupby([col_file, col_cent]).size()

    # 2. If no value: fill with "0":
    table = counts_.unstack(fill_value=0).sort_index(axis=1)
    return table

# Create mask for the verbs  
mask_verbs = df["POS"].fillna("").str.startswith("V-")

# Apply filter mask
verbs_only = df[mask_verbs]

# Apply function
freq_file_century = x_col_times_y_col(verbs_only, "File", "century")
print(freq_file_century)

century              11     12    13   14    15    16    17
File                                                       
afnik                 0      0     0    0   804     0     0
avv                   0      0     0    0     0     0  4511
birchbark             8    185    90   61    11     0     0
const                 0      0     0    0  1684     0     0
domo                  0      0     0    0     0  3589     0
drac                  0      0     0    0   546     0     0
dux-grjaz             0      0     0    0     0    34     0
kiev-hyp              0     95     0    0     0     0     0
lav                   0  11477     0    0     0     0     0
luk-koloc             0      0     0    0   152     0     0
mst                   0     23     0    0     0     0     0
mstislav-col          0     34     0    0     0     0     0
nov-list              0      0     0    0    38     0     0
nov-marg              0      0    15    0     0     0     0
nov-sin               0      0  2706    

In [21]:
df.place.unique()

array(['Novgorod', 'Pskov', 'Unknown', 'Staraja_Russa', 'Moscow',
       'Byzantine_Empire', 'Nizhny_Novgorod', 'Smolensk', 'Kiev',
       'Latvia', 'NorthernRussia', 'Tver'], dtype=object)

In [22]:
# Uncomment to write Excel file
# freq_file_century.to_excel("file_century_verbs.xlsx")

In [23]:
def groupby_col_row_size_table_optional_filter(
    df,
    col,
    row,
    filter_col=None,
    filter_val=None
):
    """
    Group df by col1 and col2 and create Pivot-Table:
        col -> returns  Table_Col, 
        row -> returns Table_Row
    
    Counts the frequencies of col1-col2 combinations
    for special filter_val in filter_col 
    (e.g. filter_col "POS" with containing filter_val "V-")
    """
    #  Flter None entries
    if filter_col and filter_val is not None:
        df = df[df[filter_col] == filter_val]

    # Create Pivot table
    counts = df.groupby([col, row]).size().unstack(fill_value=0)

    # Sort columns numerically 
    numeric = [c for c in counts.columns if str(c).isdigit()]
    others  = [c for c in counts.columns if c not in numeric]
    sorted_cols = sorted(numeric, key=lambda x: int(x)) + sorted(others)
    table = counts[sorted_cols]

    # 3) Create column "places"
    if 'place' in df.columns:
        exp = df.copy()
        # if list of "places" is empty:
        if exp['place'].apply(lambda x: isinstance(x, list)).any():
            exp = exp.explode('place')
        places = (
            exp.dropna(subset=['place'])
               .drop_duplicates(subset=[col, 'place'])
               .groupby(col)['place']
               .apply(lambda lst: sorted(lst.tolist()))
        )
        table['places'] = places

    return table

# Function Call: groupby_col_row_size_table_optional_filter

In [24]:
table_place_century = groupby_col_row_size_table_optional_filter(df, "place", "century")
print(table_place_century)


century              11     12     13    14     15     16     17  \
place                                                              
Byzantine_Empire      0      0      0     0   9258      0      0   
Kiev              25684   7568      0     0      0      0      0   
Latvia                0      0    171     0      0      0      0   
Moscow                0  23760      0  2399  22687  23880  24089   
Nizhny_Novgorod       0  56725      0     0      0      0      0   
NorthernRussia        0      0      0     0   2487      0   1835   
Novgorod            248   1487  18799   352    270      0      0   
Pskov                 0   3610     56     0   1245      0      0   
Smolensk              0      0   1421   344      0      0      0   
Staraja_Russa         0     42      0     0      0      0      0   
Tver                  0      0      0     0   6842      0      0   
Unknown               0     16      0     0      0      0      0   

century                       places  
place   

In [25]:
table = groupby_col_row_size_table_optional_filter(
    df,
    col='region',
    row='century',
    filter_col='POS',
    filter_val='V-'
)
print(table)

century             11     12    13   14    15    16    17  \
region                                                       
Byzantine_Empire     0      0     0    0  1684     0     0   
East Slavic       5145  17992  3087  455  5730  3623  4906   
Latvia               0      0    28    0     0     0     0   
Unknown              0      2     0    0     0     0     0   

century                                                      places  
region                                                               
Byzantine_Empire                                 [Byzantine_Empire]  
East Slavic       [Kiev, Moscow, Nizhny_Novgorod, NorthernRussia...  
Latvia                                                     [Latvia]  
Unknown                                                   [Unknown]  


In [26]:
# Count sum of verbs per Century (over all files)
df_century_value_count_after_filter = df.century.value_counts()
print(df_century_value_count_after_filter)

century
12    93208
15    42789
11    25932
17    25924
16    23880
13    20447
14     3095
Name: count, dtype: int64


In [27]:
# Sum of verbs by region and century
table_region_century_VERBS_ONLY = groupby_col_row_size_table_optional_filter(df, "region", "century", filter_col="POS" , filter_val="V-")
print(table_region_century_VERBS_ONLY)

century             11     12    13   14    15    16    17  \
region                                                       
Byzantine_Empire     0      0     0    0  1684     0     0   
East Slavic       5145  17992  3087  455  5730  3623  4906   
Latvia               0      0    28    0     0     0     0   
Unknown              0      2     0    0     0     0     0   

century                                                      places  
region                                                               
Byzantine_Empire                                 [Byzantine_Empire]  
East Slavic       [Kiev, Moscow, Nizhny_Novgorod, NorthernRussia...  
Latvia                                                     [Latvia]  
Unknown                                                   [Unknown]  


## Optional: Save results as tables -> comment out

In [28]:
#!pip install openpyxl

In [29]:
#table_place_century_VERBS_ONLY.to_excel("OUTPUTS/VERBS_by_PLACExCENTURY.xlsx")

In [30]:
#table_region_century_VERBS_ONLY.to_excel("OUTPUTS/VERBS_by_REGIONxCENTURY.xlsx")